# 01. Import Libraries 

In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib
import os
import folium
import json

In [2]:
%matplotlib inline

# 02. Import Data

In [3]:
#JSON file
state_geo = r'C:\Users\byl19\OneDrive\Desktop\states.json'

In [4]:
path = r'C:\Users\byl19\Acheivment 6\Acheivment 6'

In [5]:
df = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'Cleaned-Oil-Data.csv'), index_col = False)

# 03. Wrangling Data

In [6]:
df.columns

Index(['Report Number', 'Accident Year', 'Accident Date', 'Accident Time',
       'Operator ID', 'Operator Name', 'Pipeline/Facility Name',
       'Pipeline Location', 'Pipeline Type', 'Liquid Type', 'Accident City',
       'Accident County', 'Accident State', 'Cause Category',
       'Cause Subcategory', 'Unintentional Release (Barrels)',
       'Intentional Release (Barrels)', 'Liquid Recovery (Barrels)',
       'Net Loss (Barrels)', 'Liquid Ignition', 'Liquid Explosion',
       'Pipeline Shutdown', 'Shutdown Date/Time', 'Restart Date/Time',
       'Public Evacuations', 'Property Damage Costs', 'Lost Commodity Costs',
       'Public/Private Property Damage Costs', 'Emergency Response Costs',
       'Environmental Remediation Costs', 'Other Costs', 'All Costs',
       'Time Category'],
      dtype='object')

In [7]:
df['Accident State'].value_counts()

TX    1004
OK     236
LA     169
CA     153
KS     150
IL     108
WY      98
NJ      85
MN      59
IN      57
NM      57
OH      53
ND      49
IA      46
PA      45
MO      42
MS      32
MT      30
MI      29
WI      27
CO      24
NE      22
AL      21
VA      20
GA      17
SC      16
NC      15
NY      14
KY      14
AR      13
AK      11
SD       9
UT       8
MD       8
WA       8
FL       7
TN       7
ID       4
OR       4
HI       3
WV       2
CT       2
MA       2
PR       1
ME       1
NV       1
Name: Accident State, dtype: int64

#### Our states are in abbreviated format which is different than our JSON file. We will need to switch that so the two can work together.

In [8]:
#states dictionary
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

In [9]:
#replacing states
df['Accident State'] = df['Accident State'].replace(states)

In [10]:
#checking states
df['Accident State'].head()

0    West Virginia
1            Texas
2        Louisiana
3         Illinois
4     North Dakota
Name: Accident State, dtype: object

In [11]:
#making plottable dataframe
data = pd.DataFrame(df['Accident State'].value_counts().reset_index().values, columns=["State", "Count of Spills"])
dataindex = data.sort_index(axis = 0, ascending=True)
dataindex

,State,Count of Spills
0,Texas,1004
1,Oklahoma,236
2,Louisiana,169
3,California,153
4,Kansas,150
5,Illinois,108
6,Wyoming,98
7,New Jersey,85
8,Minnesota,59
9,Indiana,57


In [12]:
#thresholds for our legand
ticks = [1,25,100,250,1004]

In [13]:
#creating map
statemap = folium.Map(location = [60, -100], zoom_start = 1.5)

folium.Choropleth(
    geo_data = state_geo, 
    data = dataindex,
    columns = ['State', 'Count of Spills'],
    key_on = 'feature.properties.name', 
    threshold_scale = ticks,
    fill_color = 'GnBu', fill_opacity=0.6, line_opacity=0.1,
    legend_name = "Count of Spills").add_to(statemap)
folium.LayerControl().add_to(statemap)

statemap

#### We have the map with our states, I made the ticks a custom distance apart since the highest value is so much bigger than the second highest and now we can clearly see the data for all states. To see a more specific area of the spills, sorting them by counties will give us more specific locations

In [15]:
#save map
statemap.save('State_Data.html')

In [28]:
county_geo = r'C:\Users\byl19\OneDrive\Desktop\counties.json'

In [17]:
df['Accident County'].value_counts()

HARRIS            167
JEFFERSON         106
LOS ANGELES        52
PAYNE              50
MIDLAND            49
                 ... 
REFUGIO COUNTY      1
ARAPAHOE            1
FAYETTE             1
ADAMS COUNTY        1
GONZALES            1
Name: Accident County, Length: 678, dtype: int64

In [18]:
#Counties are in all caps which is different from geojson file

In [19]:
df['Accident County'] = df['Accident County'].str.lower()

In [20]:
df['Accident County'] = df['Accident County'].str.title()

#### I lowercased everything and then used the command .title becasue just capitalizing made the counties with 2 words in their name incorrect as the first word was capitalizd but the second was not. Lower casing everything and then using .title capitalized both words in the name.

In [21]:
df['Accident County'].value_counts()

Harris            167
Jefferson         106
Los Angeles        52
Payne              50
Midland            49
                 ... 
Refugio County      1
Arapahoe            1
Fayette             1
Adams County        1
Gonzales            1
Name: Accident County, Length: 678, dtype: int64

In [24]:
#Matching JSON file
df['Accident County'] = df['Accident County'].str.replace(' County', '')

In [25]:
#create plottable datafrom for counties
counties = pd.DataFrame(df['Accident County'].value_counts().reset_index().values, columns=("County", "Count of Spills"))
countiesindex = counties.sort_index(axis = 0, ascending=True)

In [26]:
countiesindex

,County,Count of Spills
0,Harris,169
1,Jefferson,107
2,Los Angeles,56
3,Payne,51
4,Midland,49
...,...,...
621,La Plata,1
622,Miami,1
623,Bannock,1
624,Clarke,1


In [36]:
#threshold for counties legand
ticks2 = [1,10,30,85,150,200]

In [51]:
#counties map
countiesmap = folium.Map(location = [40, -100], zoom_start = 3.3)

folium.Choropleth(
    geo_data = county_geo, 
    data = countiesindex,
    columns = ('County', 'Count of Spills'),
    key_on = 'feature.properties.name', 
    threshold_scale = ticks2,
    fill_color = 'GnBu', fill_opacity=0.6, line_opacity=0.1,
    legend_name = "spills").add_to(countiesmap)
folium.LayerControl().add_to(countiesmap)

countiesmap

#### There are a good amount of counties that do not have a documented oil spill from 2010 to 2017. I would love to find updated data on this so my map could be more updated. This map does however give a good picture on the counties who have had a lot of oil spills, specifically Harris county in east texas. I make the ticks for the legand custom to really show what counties had a lot of spills and what counties didnt. 

In [52]:
#save map
countiesmap.save('County_Data_Map.html')

#### These maps really help us answer the question, "in what location are most of these spills happening?". We have a broad overview of the spills looking at it by state and we can pinpoint the spills better by looking at which counties they occured in. For me, this brought up the question of are there more spills in each state/county because they have more pipelines/pipline compnaies? SOmething I could do next is find data on the number of piplines within each state/county and get the proportion of pipelines that have a spill in relation to the total number.

In [53]:
#export data
df.to_csv(r'C:\Users\byl19\Acheivment 6\Acheivment 6\02 Data\Prepared Data\Cleaned-Oil-Data.csv', index = False)